In [52]:
import dask
import dask.dataframe as df
import numpy as np
import pandas as pd
from scipy import sparse as sp
from math import log
from collections import Counter
import functools 
import sklearn.metrics as metrics
from itertools import combinations, product
from string import ascii_lowercase
np.random.seed(0)

In [2]:
pandas_df = pd.DataFrame(data={'col1':[1,2,3,3,2,1], 'col2':[4,5,4,4,5,5], 'col3':[3,4,3,4,3,4],
                                            'col4':[1,2,3,4,5,6], 'col5':[3,3,3,3,3,3], 'cols':['a','b','c','d','e','f']})

In [3]:
dask_df = df.from_pandas(pandas_df,npartitions=2)

In [48]:
attributes = ['col1', 'col2','col3','col4']
data = dask_df.copy()[attributes]
data['count'] = 1
stats = data.groupby(attributes).sum()

AttributeError: 'function' object has no attribute 'compute'

In [20]:
pandas_stats = pandas_df.copy().loc[:,attributes]
pandas_stats['count']=1
pandas_stats = pandas_stats.groupby(attributes).sum()
pandas_stats

count
col1 col2 col3 col4       
1    4    3    1         1
     5    4    6         1
2    5    3    5         1
          4    2         1
3    4    3    3         1
          4    4         1

In [21]:
full_space = pd.DataFrame(columns=attributes, data=list(product(*pandas_stats.index.levels)))

In [22]:
full_space

,col1,col2,col3,col4
0,1,4,3,1
1,1,4,3,2
2,1,4,3,3
3,1,4,3,4
4,1,4,3,5
5,1,4,3,6
6,1,4,4,1
7,1,4,4,2
8,1,4,4,3
9,1,4,4,4


In [13]:
pandas_stats.reset_index(inplace=True)
pandas_stats

,col1,col2,count
0,1,4,1
1,1,5,1
2,2,5,2
3,3,4,2


In [14]:
pandas_stats = pd.merge(full_space, pandas_stats, how='left')
pandas_stats

,col1,col2,count
0,1,4,1.0
1,1,5,1.0
2,2,4,NaN
3,2,5,2.0
4,3,4,2.0
5,3,5,NaN


In [15]:
pandas_stats.fillna(0, inplace=True)

In [16]:
pandas_stats

,col1,col2,count
0,1,4,1.0
1,1,5,1.0
2,2,4,0.0
3,2,5,2.0
4,3,4,2.0
5,3,5,0.0


In [23]:
f = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
   ...:                           'foo', 'bar', 'foo', 'foo'],
   ...:                    'B' : ['one', 'one', 'two', 'three',
   ...:                           'two', 'two', 'one', 'three'],
   ...:                    'C' : np.random.randn(8),
   ...:                    'D' : np.random.randn(8)})

In [24]:
f

,A,B,C,D
0,foo,one,1.764052,-0.103219
1,bar,one,0.400157,0.410599
2,foo,two,0.978738,0.144044
3,bar,three,2.240893,1.454274
4,foo,two,1.867558,0.761038
5,bar,two,-0.977278,0.121675
6,foo,one,0.950088,0.443863
7,foo,three,-0.151357,0.333674


In [26]:
grouped = f.groupby(['A', 'B'])

In [29]:
grouped=grouped.sum()

In [30]:
grouped.index.levels

FrozenList([['bar', 'foo'], ['one', 'three', 'two']])

In [31]:
grouped

C         D
A   B                        
bar one    0.400157  0.410599
    three  2.240893  1.454274
    two   -0.977278  0.121675
foo one    2.714141  0.340644
    three -0.151357  0.333674
    two    2.846296  0.905081

In [32]:
list(product(*grouped.index.levels))

[('bar', 'one'),
 ('bar', 'three'),
 ('bar', 'two'),
 ('foo', 'one'),
 ('foo', 'three'),
 ('foo', 'two')]

In [50]:
np.random.randint(1, 10)

6

In [54]:
''.join(np.random.choice(list(ascii_lowercase), size=6))

'hjtvse'

In [56]:
np.random.uniform(0, 100, 10)

array([29.75346065,  5.67129773, 27.26562946, 47.76651173, 81.21687288,
       47.99771724, 39.27847961, 83.60787635, 33.73961604, 64.81718721])